In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [ ]:
# Additional dependencies for this notebook
!pip install -q qiskit-addon-obp

*تقدير الاستخدام: 16 دقيقة على معالج Eagle r3 (ملاحظة: هذا تقدير فحسب. قد يختلف وقت التشغيل الفعلي لديك.)*

In [ ]:
# This cell is hidden from users;
# it disables linting rules.
# ruff: noqa

## الخلفية النظرية

الانتشار الخلفي للمؤثرات هو أسلوب يقوم على استيعاب العمليات من نهاية الدائرة الكمية في المؤثر المقاس، مما يُقلّص عمق الدائرة بشكل عام على حساب زيادة عدد الحدود في المؤثر. والهدف هو إعادة الانتشار الخلفي لأكبر قدر ممكن من الدائرة دون السماح للمؤثر بالنمو بشكل مفرط. يتوفر تطبيق قائم على Qiskit في إضافة OBP الخاصة بـ Qiskit، ويمكن الاطلاع على مزيد من التفاصيل في [التوثيق المقابل](/guides/qiskit-addons-obp) مع [مثال مبسّط](/guides/qiskit-addons-obp-get-started) للبدء.

لنأخذ مثالاً لدائرة يُراد فيها قياس المؤثر $O = \sum_P c_P P$، حيث $P$ هي عوامل Pauli و $c_P$ هي المعاملات. لنُرمز للدائرة بالوحدوية الأحادية $U$ التي يمكن تقسيمها منطقياً إلى $U = U_C U_Q$ كما هو موضح في الشكل أدناه.

![Circuit diagram showing Uq followed by Uc](../docs/images/tutorials/improving-estimation-of-expectation-values-with-operator-backpropagation/logical-partitioning.avif)

يستوعب الانتشار الخلفي للمؤثرات الوحدوية $U_C$ في المؤثر عن طريق تطويرها على النحو $O' = U_C^{\dagger}OU_C = \sum_P c_P U_C^{\dagger}PU_C$. بمعنى آخر، يُنفَّذ جزء من الحساب كلاسيكياً عبر تطور المؤثر من $O$ إلى $O'$. يمكن الآن إعادة صياغة المسألة الأصلية على أنها قياس المؤثر $O'$ للدائرة ذات العمق الأصغر التي وحدويتها $U_Q$.

تُمثَّل الوحدوية $U_C$ كعدد من الشرائح $U_C = U_S U_{S-1}...U_2U_1$. وتوجد طرق متعددة لتعريف الشريحة؛ فعلى سبيل المثال في الدائرة أعلاه، يمكن اعتبار كل طبقة من بوابات $R_{zz}$ وكل طبقة من بوابات $R_x$ شريحةً مستقلة. يشمل الانتشار الخلفي حساب $O' = \Pi_{s=1}^S \sum_P c_P U_s^{\dagger} P U_s$ كلاسيكياً. يمكن تمثيل كل شريحة $U_s$ على النحو $U_s = exp(\frac{-i\theta_s P_s}{2})$، حيث $P_s$ هو عامل Pauli على $n$ كيوبت و $\theta_s$ هو قيمة عددية. ومن السهل التحقق من أن:

$$
U_s^{\dagger} P U_s = P \qquad \text{if} ~[P,P_s] = 0,
$$
$$
U_s^{\dagger} P U_s = \qquad cos(\theta_s)P + i sin(\theta_s)P_sP \qquad \text{if} ~{P,P_s} = 0
$$

في المثال أعلاه، إذا كان ${P,P_s} = 0$، فإننا نحتاج إلى تنفيذ دائرتين كميتين بدلاً من واحدة لحساب قيمة التوقع. لذا، قد يزيد الانتشار الخلفي عدد الحدود في المؤثر، مما يؤدي إلى عدد أكبر من عمليات تنفيذ الدوائر. إحدى الطرق للسماح بانتشار خلفي أعمق في الدائرة مع منع نمو المؤثر بشكل مفرط هي اقتطاع الحدود ذات المعاملات الصغيرة بدلاً من إضافتها إلى المؤثر. فعلى سبيل المثال في المثال أعلاه، قد يُختار اقتطاع الحد المتضمن $P_sP$ شريطة أن يكون $\theta_s$ صغيراً بما يكفي. يؤدي اقتطاع الحدود إلى تقليل عدد الدوائر الكمية الواجب تنفيذها، غير أن ذلك يُحدث خطأً في حساب قيمة التوقع النهائية يتناسب مع حجم معاملات الحدود المقتطعة.

يُنفّذ هذا الدليل التعليمي [نمط Qiskit](/guides/intro-to-patterns) لمحاكاة الديناميكا الكمية لسلسلة دوران Heisenberg باستخدام <a href="https://github.com/Qiskit/qiskit-addon-obp">qiskit-addon-obp</a>.

## المتطلبات

قبل البدء في هذا الدليل التعليمي، تأكد من تثبيت ما يلي:

- Qiskit SDK الإصدار 1.2 أو أحدث (`pip install qiskit`)
- Qiskit Runtime الإصدار 0.28 أو أحدث (`pip install qiskit-ibm-runtime`)
- إضافة OBP الخاصة بـ Qiskit (`pip install qiskit-addon-obp`)
- أدوات إضافية لـ Qiskit (`pip install qiskit-addon-utils`)

## الإعداد

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit.primitives import StatevectorEstimator as Estimator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import CouplingMap
from qiskit.synthesis import LieTrotter

from qiskit_addon_utils.problem_generators import generate_xyz_hamiltonian
from qiskit_addon_utils.problem_generators import (
    generate_time_evolution_circuit,
)
from qiskit_addon_utils.slicing import slice_by_gate_types, combine_slices
from qiskit_addon_obp.utils.simplify import OperatorBudget
from qiskit_addon_obp import backpropagate
from qiskit_addon_obp.utils.truncating import setup_budget

from rustworkx.visualization import graphviz_draw

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2, EstimatorOptions

## الجزء الأول: سلسلة دوران Heisenberg صغيرة الحجم
### الخطوة 1: تحويل المدخلات الكلاسيكية إلى مسألة كمية
#### تحويل التطور الزمني لنموذج Heisenberg الكمي إلى تجربة كمية.
توفر حزمة [qiskit_addon_utils](https://github.com/Qiskit/qiskit-addon-utils) وظائف قابلة لإعادة الاستخدام لأغراض متعددة.

يوفر وحدتها [qiskit_addon_utils.problem_generators](https://docs.quantum.ibm.com/api/qiskit-addon-utils/problem-generators) دوالَّ لتوليد هاملتونيات شبيهة بـ Heisenberg على رسم بياني لشبكة اتصال محددة.
يمكن أن يكون هذا الرسم البياني إما [rustworkx.PyGraph](https://www.rustworkx.org/apiref/rustworkx.PyGraph.html) أو [CouplingMap](https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.CouplingMap)، مما يجعل استخدامه سلساً في سير عمل Qiskit.

فيما يلي، نُنشئ خريطة اقتران `CouplingMap` خطية مكوّنة من 10 كيوبتات.

In [3]:
num_qubits = 10
layout = [(i - 1, i) for i in range(1, num_qubits)]

# Instantiate a CouplingMap object
coupling_map = CouplingMap(layout)
graphviz_draw(coupling_map.graph, method="circo")

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/de8ce35e-a2c5-474f-adb9-88c9afb2bd76-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/operator-back-propagation/extracted-outputs/de8ce35e-a2c5-474f-adb9-88c9afb2bd76-0.avif)

بعد ذلك، نُولّد مؤثر Pauli يُمثّل هاملتونيان Heisenberg XYZ.

$$
{\hat{\mathcal{H}}_{XYZ} = \sum_{(j,k)\in E} (J_{x} \sigma_j^{x} \sigma_{k}^{x} + J_{y} \sigma_j^{y} \sigma_{k}^{y} + J_{z} \sigma_j^{z} \sigma_{k}^{z}) + \sum_{j\in V} (h_{x} \sigma_j^{x} + h_{y} \sigma_j^{y} + h_{z} \sigma_j^{z})}
$$

حيث $G(V,E)$ هو الرسم البياني لخريطة الاقتران المُعطاة.

In [4]:
# Get a qubit operator describing the Heisenberg XYZ model
hamiltonian = generate_xyz_hamiltonian(
    coupling_map,
    coupling_constants=(np.pi / 8, np.pi / 4, np.pi / 2),
    ext_magnetic_field=(np.pi / 3, np.pi / 6, np.pi / 9),
)
print(hamiltonian)

SparsePauliOp(['IIIIIIIXXI', 'IIIIIIIYYI', 'IIIIIIIZZI', 'IIIIIXXIII', 'IIIIIYYIII', 'IIIIIZZIII', 'IIIXXIIIII', 'IIIYYIIIII', 'IIIZZIIIII', 'IXXIIIIIII', 'IYYIIIIIII', 'IZZIIIIIII', 'IIIIIIIIXX', 'IIIIIIIIYY', 'IIIIIIIIZZ', 'IIIIIIXXII', 'IIIIIIYYII', 'IIIIIIZZII', 'IIIIXXIIII', 'IIIIYYIIII', 'IIIIZZIIII', 'IIXXIIIIII', 'IIYYIIIIII', 'IIZZIIIIII', 'XXIIIIIIII', 'YYIIIIIIII', 'ZZIIIIIIII', 'IIIIIIIIIX', 'IIIIIIIIIY', 'IIIIIIIIIZ', 'IIIIIIIIXI', 'IIIIIIIIYI', 'IIIIIIIIZI', 'IIIIIIIXII', 'IIIIIIIYII', 'IIIIIIIZII', 'IIIIIIXIII', 'IIIIIIYIII', 'IIIIIIZIII', 'IIIIIXIIII', 'IIIIIYIIII', 'IIIIIZIIII', 'IIIIXIIIII', 'IIIIYIIIII', 'IIIIZIIIII', 'IIIXIIIIII', 'IIIYIIIIII', 'IIIZIIIIII', 'IIXIIIIIII', 'IIYIIIIIII', 'IIZIIIIIII', 'IXIIIIIIII', 'IYIIIIIIII', 'IZIIIIIIII', 'XIIIIIIIII', 'YIIIIIIIII', 'ZIIIIIIIII'],
              coeffs=[0.39269908+0.j, 0.78539816+0.j, 1.57079633+0.j, 0.39269908+0.j,
 0.78539816+0.j, 1.57079633+0.j, 0.39269908+0.j, 0.78539816+0.j,
 1.57079633+0.j, 0.39269908+0.j, 0.

From the qubit operator, we can generate a quantum circuit which models its time evolution.
Once again, the [qiskit_addon_utils.problem_generators](/docs/api/qiskit-addon-utils/problem-generators) module comes to the rescue with a handy function do just that:

In [5]:
circuit = generate_time_evolution_circuit(
    hamiltonian,
    time=0.2,
    synthesis=LieTrotter(reps=2),
)
circuit.draw("mpl", style="iqp", scale=0.6)

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/1d68f197-ffa4-49de-9fe8-243b1facbd00-0.avif" alt="Output of the previous code cell" />

انطلاقاً من مؤثر الكيوبت، يمكننا توليد دائرة كمية تُنمذج تطوره الزمني.
وتُتيح وحدة [qiskit_addon_utils.problem_generators](https://docs.quantum.ibm.com/api/qiskit-addon-utils/problem-generators) دالةً مناسبة تؤدي هذا الغرض تحديداً:

In [6]:
slices = slice_by_gate_types(circuit)
print(f"Separated the circuit into {len(slices)} slices.")

Separated the circuit into 18 slices.


![Output of the previous code cell](../docs/images/tutorials/operator-back-propagation/extracted-outputs/1d68f197-ffa4-49de-9fe8-243b1facbd00-0.avif)

### الخطوة 2: تحسين المسألة لتنفيذها على العتاد الكمي
#### إنشاء شرائح الدائرة لإجراء الانتشار الخلفي
تذكّر أن دالة ``backpropagate`` ستُعيد الانتشار الخلفي لشرائح كاملة في كل مرة، لذا فإن طريقة تقسيم الدائرة إلى شرائح قد تؤثر في كفاءة الانتشار الخلفي لمسألة بعينها. هنا سنُجمّع البوابات من النوع نفسه في شرائح باستخدام دالة [slice_by_gate_types](https://docs.quantum.ibm.com/api/qiskit-addon-utils/slicing#slice_by_gate_types).

للاطلاع على نقاش أكثر تفصيلاً حول تقسيم الدوائر إلى شرائح، راجع [دليل كيفية](https://qiskit.github.io/qiskit-addon-utils/how_tos/create_circuit_slices.html) الخاص بحزمة [qiskit-addon-utils](https://github.com/Qiskit/qiskit-addon-utils).

In [7]:
op_budget = OperatorBudget(max_qwc_groups=8)

#### Backpropagate slices from the circuit

First we specify the observable to be $M_Z = \frac{1}{N} \sum_{i=1}^N \langle Z_i \rangle$, $N$ being the number of qubits. We will backpropagate slices from the time-evolution circuit until the terms in the observable can no longer be combined into eight or fewer qubit-wise commuting Pauli groups.

In [8]:
observable = SparsePauliOp.from_sparse_list(
    [("Z", [i], 1 / num_qubits) for i in range(num_qubits)],
    num_qubits=num_qubits,
)
observable

SparsePauliOp(['IIIIIIIIIZ', 'IIIIIIIIZI', 'IIIIIIIZII', 'IIIIIIZIII', 'IIIIIZIIII', 'IIIIZIIIII', 'IIIZIIIIII', 'IIZIIIIIII', 'IZIIIIIIII', 'ZIIIIIIIII'],
              coeffs=[0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j,
 0.1+0.j, 0.1+0.j])

#### تقييد الحد الأقصى لنمو المؤثر خلال الانتشار الخلفي
خلال الانتشار الخلفي، سيتجه عدد الحدود في المؤثر عموماً نحو $4^N$ بسرعة، حيث $N$ هو عدد الكيوبتات. عندما لا يتبادل حدّان في المؤثر التبديل على مستوى الكيوبتات، نحتاج إلى دوائر منفصلة للحصول على قيم التوقع المقابلة لكل منهما. فعلى سبيل المثال، إذا كان لدينا مؤثر يعمل على كيوبتين $O = 0.1 XX + 0.3 IZ - 0.5 IX$، فبما أن $[XX,IX] = 0$، يكفي قياس واحد في أساس واحد لحساب قيم التوقع لهذين الحدّين. غير أن $IZ$ يُضاد التبادل مع الحدّين الآخرين، لذا نحتاج إلى قياس في أساس منفصل لحساب قيمة التوقع لـ $IZ$. بمعنى آخر، نحتاج إلى دائرتين بدلاً من واحدة لحساب $\langle O \rangle$. مع تزايد عدد الحدود في المؤثر، يرتفع احتمال زيادة العدد المطلوب من تنفيذات الدائرة أيضاً.

يمكن تحديد حد أقصى لحجم المؤثر عبر تمرير الوسيط ``operator_budget`` إلى دالة ``backpropagate``، الذي يقبل نسخة من [OperatorBudget](https://docs.quantum.ibm.com/api/qiskit-addon-obp/utils-simplify#operatorbudget).

للتحكم في الموارد الإضافية (الوقت) المُخصَّصة، نحدّد الحد الأقصى لعدد مجموعات Pauli المتبادلة كيوبتياً التي يُسمح للمؤثر المُعاد انتشاره الخلفي بامتلاكها. هنا نُحدّد أن الانتشار الخلفي يجب أن يتوقف حين يتجاوز عدد مجموعات Pauli المتبادلة كيوبتياً في المؤثر الحد 8.

In [9]:
# Backpropagate slices onto the observable
bp_obs, remaining_slices, metadata = backpropagate(
    observable, slices, operator_budget=op_budget
)
# Recombine the slices remaining after backpropagation
bp_circuit = combine_slices(remaining_slices)

print(f"Backpropagated {metadata.num_backpropagated_slices} slices.")
print(
    f"New observable has {len(bp_obs.paulis)} terms, which can be combined into {len(bp_obs.group_commuting(qubit_wise=True))} groups."
)
print(
    f"Note that backpropagating one more slice would result in {metadata.backpropagation_history[-1].num_paulis[0]} terms "
    f"across {metadata.backpropagation_history[-1].num_qwc_groups} groups."
)
print("The remaining circuit after backpropagation looks as follows:")
bp_circuit.draw("mpl", fold=-1, scale=0.6)

Backpropagated 6 slices.
New observable has 60 terms, which can be combined into 6 groups.
Note that backpropagating one more slice would result in 114 terms across 12 groups.
The remaining circuit after backpropagation looks as follows:


<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/65ec9cb1-a4ed-497b-a616-180e9659956f-1.avif" alt="Output of the previous code cell" />

#### إعادة الانتشار الخلفي للشرائح من الدائرة
أولاً نُحدّد المؤثر المقاس على أنه $M_Z = \frac{1}{N} \sum_{i=1}^N \langle Z_i \rangle$، حيث $N$ هو عدد الكيوبتات. سنُعيد الانتشار الخلفي للشرائح من دائرة التطور الزمني حتى لا تتمكن حدود المؤثر من الاندماج ضمن ثماني مجموعات أو أقل من مجموعات Pauli المتبادلة كيوبتياً.

In [10]:
truncation_error_budget = setup_budget(max_error_per_slice=0.005)

Note that by allocating `5e-3` error per slice for truncation, we are able to remove 1 more slice from the circuit, while remaining within the original budget of eight commuting Pauli groups in the observable. By default, `backpropagate` uses the L1 norm of the truncated coefficients to bound the total error incurred from truncation. For other options refer to the [how-to guide on specifying the p_norm](https://qiskit.github.io/qiskit-addon-obp/how_tos/bound_error_using_p_norm.html).

In this particular example where we have backpropagated seven slices, the total truncation error should not exceed ``(5e-3 error/slice) * (7 slices) = 3.5e-2``.
For further discussion on distributing an error budget across your slices, check out [this how-to guide](https://qiskit.github.io/qiskit-addon-obp/how_tos/truncate_operator_terms.html).

In [11]:
# Run the same experiment but truncate observable terms with small coefficients
bp_obs_trunc, remaining_slices_trunc, metadata = backpropagate(
    observable,
    slices,
    operator_budget=op_budget,
    truncation_error_budget=truncation_error_budget,
)

# Recombine the slices remaining after backpropagation
bp_circuit_trunc = combine_slices(
    remaining_slices_trunc, include_barriers=False
)

print(f"Backpropagated {metadata.num_backpropagated_slices} slices.")
print(
    f"New observable has {len(bp_obs_trunc.paulis)} terms, which can be combined into {len(bp_obs_trunc.group_commuting(qubit_wise=True))} groups.\n"
    f"After truncation, the error in our observable is bounded by {metadata.accumulated_error(0):.3e}"
)
print(
    f"Note that backpropagating one more slice would result in {metadata.backpropagation_history[-1].num_paulis[0]} terms "
    f"across {metadata.backpropagation_history[-1].num_qwc_groups} groups."
)
print("The remaining circuit after backpropagation looks as follows:")
bp_circuit_trunc.draw("mpl", scale=0.6)

Backpropagated 7 slices.
New observable has 82 terms, which can be combined into 8 groups.
After truncation, the error in our observable is bounded by 3.266e-02
Note that backpropagating one more slice would result in 114 terms across 12 groups.
The remaining circuit after backpropagation looks as follows:


<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/5e8bae1a-ef18-4eb0-9d2a-1ac7bbdced3b-1.avif" alt="Output of the previous code cell" />

ستلاحظ فيما يلي أننا أعدنا الانتشار الخلفي لست شرائح، وأن الحدود اندمجت في ست مجموعات لا ثماني. وهذا يعني أن إعادة الانتشار الخلفي لشريحة واحدة إضافية ستتجاوز حد الثماني مجموعات من Pauli. يمكننا التحقق من ذلك بفحص البيانات الوصفية المُعادة. لاحظ أيضاً أن تحويل الدائرة في هذا الجزء دقيق تماماً، أي أنه لم يُقتطع أي حد من المؤثر الجديد $O'$. الدائرة والمؤثر اللذان خضعا للانتشار الخلفي يُعطيان النتيجة ذاتها التي تُعطيها الدائرة والمؤثر الأصليان.

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)

In [13]:
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)

# Transpile original experiment
circuit_isa = pm.run(circuit)
observable_isa = observable.apply_layout(circuit_isa.layout)

# Transpile backpropagated experiment
bp_circuit_isa = pm.run(bp_circuit)
bp_obs_isa = bp_obs.apply_layout(bp_circuit_isa.layout)

# Transpile the backpropagated experiment with truncated observable terms
bp_circuit_trunc_isa = pm.run(bp_circuit_trunc)
bp_obs_trunc_isa = bp_obs_trunc.apply_layout(bp_circuit_trunc_isa.layout)

![Output of the previous code cell](../docs/images/tutorials/operator-back-propagation/extracted-outputs/65ec9cb1-a4ed-497b-a616-180e9659956f-1.avif)

بعد ذلك، سنُحدّد المسألة ذاتها بالقيود نفسها على حجم المؤثر الناتج. غير أننا هذه المرة سنُخصّص ميزانية خطأ لكل شريحة باستخدام دالة [setup_budget](https://docs.quantum.ibm.com/api/qiskit-addon-obp/utils-truncating#setup_budget). ستُقتطع حدود Pauli ذات المعاملات الصغيرة من كل شريحة حتى تمتلئ ميزانية الخطأ، وستُنقل الميزانية المتبقية إلى الشريحة التالية. لاحظ أن التحويل الناتج عن الانتشار الخلفي في هذه الحالة تقريبي، نظراً لاقتطاع بعض الحدود من المؤثر.

لتفعيل هذا الاقتطاع، نحتاج إلى إعداد ميزانية الخطأ على النحو التالي:

In [14]:
pub = (circuit_isa, observable_isa)
bp_pub = (bp_circuit_isa, bp_obs_isa)
bp_trunc_pub = (bp_circuit_trunc_isa, bp_obs_trunc_isa)

لاحظ أنه بتخصيص خطأ قدره `5e-3` لكل شريحة للاقتطاع، أصبح بإمكاننا إزالة شريحة إضافية من الدائرة مع البقاء ضمن الميزانية الأصلية البالغة ثماني مجموعات Pauli المتبادلة في المؤثر. بشكل افتراضي، تستخدم `backpropagate` معيار L1 لمعاملات الحدود المقتطعة لتحديد الحد الأقصى للخطأ الكلي الناتج عن الاقتطاع. للاطلاع على خيارات أخرى راجع [دليل كيفية تحديد p_norm](https://qiskit.github.io/qiskit-addon-obp/how_tos/bound_error_using_p_norm.html).

في هذا المثال بالذات حيث أعدنا الانتشار الخلفي لسبع شرائح، يجب ألا يتجاوز إجمالي خطأ الاقتطاع ``(5e-3 error/slice) * (7 slices) = 3.5e-2``.
للمزيد من النقاش حول توزيع ميزانية الخطأ على الشرائح، راجع [هذا الدليل](https://qiskit.github.io/qiskit-addon-obp/how_tos/truncate_operator_terms.html).

In [15]:
ideal_estimator = Estimator()

# Run the experiments using Estimator primitive to obtain the exact outcome
result_exact = (
    ideal_estimator.run([(circuit, observable)]).result()[0].data.evs.item()
)
print(f"Exact expectation value: {result_exact}")

Exact expectation value: 0.8871244838989416


We shall use <a href="/docs/guides/configure-error-mitigation">resilience_level</a> = 2 for this example.

In [ ]:
options = EstimatorOptions()
options.default_precision = 0.011
options.resilience_level = 2

estimator = EstimatorV2(mode=backend, options=options)

In [ ]:
job = estimator.run([pub, bp_pub, bp_trunc_pub])

### Step 4: Post-process and return result to desired classical format

In [ ]:
result_no_bp = job.result()[0].data.evs.item()
result_bp = job.result()[1].data.evs.item()
result_bp_trunc = job.result()[2].data.evs.item()

std_no_bp = job.result()[0].data.stds.item()
std_bp = job.result()[1].data.stds.item()
std_bp_trunc = job.result()[2].data.stds.item()

In [ ]:
print(
    f"Expectation value without backpropagation: {result_no_bp} ± {std_no_bp}"
)
print(f"Backpropagated expectation value: {result_bp} ± {std_bp}")
print(
    f"Backpropagated expectation value with truncation: {result_bp_trunc} ± {std_bp_trunc}"
)

Expectation value without backpropagation: 0.8033194665993642
Backpropagated expectation value: 0.8599808781259016
Backpropagated expectation value with truncation: 0.8868736004169483


In [ ]:
methods = [
    "No backpropagation",
    "Backpropagation",
    "Backpropagation w/ truncation",
]
values = [result_no_bp, result_bp, result_bp_trunc]
stds = [std_no_bp, std_bp, std_bp_trunc]

ax = plt.gca()
plt.bar(methods, values, color="#a56eff", width=0.4, edgecolor="#8a3ffc")
plt.axhline(result_exact)
ax.set_ylim([0.6, 0.92])
plt.text(0.2, 0.895, "Exact result")
ax.set_ylabel(r"$M_Z$", fontsize=12)

Text(0, 0.5, '$M_Z$')

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/b444d8bc-c800-4aa3-9927-eb807e92194f-1.avif" alt="Output of the previous code cell" />

## Part B: Scale it up!

Let us now use Operator Backpropagation to study the dynamics of the Hamiltonian of a 50-qubit Heisenberg Spin Chain.

### Step 1: Map classical inputs to a quantum problem

We consider a 50-qubit Hamiltonian $\hat{\mathcal{H}}_{XYZ}$ for the scaled up problem with the same values for the $J$ and $h$ coefficients as in the small-scale example. The observable $M_Z = \frac{1}{N} \sum_{i=1}^N \langle Z_i \rangle$ is also the same as before. This problem is beyond classical brute-force simulation.

In [16]:
num_qubits = 50
layout = [(i - 1, i) for i in range(1, num_qubits)]

# Instantiate a CouplingMap object
coupling_map = CouplingMap(layout)
graphviz_draw(coupling_map.graph, method="circo")

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/47cb1ac7-44db-4f96-b49b-e889a920d83c-0.avif" alt="Output of the previous code cell" />

In [17]:
hamiltonian = generate_xyz_hamiltonian(
    coupling_map,
    coupling_constants=(np.pi / 8, np.pi / 4, np.pi / 2),
    ext_magnetic_field=(np.pi / 3, np.pi / 6, np.pi / 9),
)
print(hamiltonian)

SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXI', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYI', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZI', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIIIIIIIII', 'IIIIIIIIIIII

In [18]:
observable = SparsePauliOp.from_sparse_list(
    [("Z", [i], 1 / num_qubits) for i in range(num_qubits)],
    num_qubits,
)
observable

SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZ', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZI', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIIII', 'IIIIIIIIIIII

### الخطوة 4: المعالجة اللاحقة وإعادة النتيجة إلى الصيغة الكلاسيكية المطلوبة

In [19]:
circuit = generate_time_evolution_circuit(
    hamiltonian,
    time=0.2,
    synthesis=LieTrotter(reps=4),
)
circuit.draw("mpl", style="iqp", fold=-1, scale=0.6)

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/b10d16cf-95da-42c0-9b47-b2e5a8516c82-0.avif" alt="Output of the previous code cell" />

### Step 2: Optimize problem for quantum hardware execution

In [20]:
slices = slice_by_gate_types(circuit)
print(f"Separated the circuit into {len(slices)} slices.")

Separated the circuit into 36 slices.


We specify the `max_error_per_slice` to be 0.005 as before. However, since the number of slices for this large-scale problem is much higher than the small scale problem, allowing an error of 0.005 per slice may end up creating a large overall backpropagation error. We can bound this by specifying `max_error_total` which bounds the total backpropagation error, and we set its value to 0.03 (which is roughly the same as in the small-scale example).

For this large-scale example, we allow a higher value for the number of commuting groups, and set it to 15.

In [21]:
op_budget = OperatorBudget(max_qwc_groups=15)
truncation_error_budget = setup_budget(
    max_error_total=0.03, max_error_per_slice=0.005
)

Let us first obtain the backpropagated circuit and observable without any truncation.

In [22]:
bp_obs, remaining_slices, metadata = backpropagate(
    observable, slices, operator_budget=op_budget
)
bp_circuit = combine_slices(remaining_slices)

print(f"Backpropagated {metadata.num_backpropagated_slices} slices.")
print(
    f"New observable has {len(bp_obs.paulis)} terms, which can be combined into {len(bp_obs.group_commuting(qubit_wise=True))} groups."
)
print(
    f"Note that backpropagating one more slice would result in {metadata.backpropagation_history[-1].num_paulis[0]} terms "
    f"across {metadata.backpropagation_history[-1].num_qwc_groups} groups."
)
print("The remaining circuit after backpropagation looks as follows:")
bp_circuit.draw("mpl", fold=-1, scale=0.6)

Backpropagated 7 slices.
New observable has 634 terms, which can be combined into 12 groups.
Note that backpropagating one more slice would result in 1246 terms across 27 groups.
The remaining circuit after backpropagation looks as follows:


<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/164e2f00-25b6-4cf1-98f8-8b2886f007ee-1.avif" alt="Output of the previous code cell" />

Now allowing for truncation, we obtain:

In [23]:
bp_obs_trunc, remaining_slices_trunc, metadata = backpropagate(
    observable,
    slices,
    operator_budget=op_budget,
    truncation_error_budget=truncation_error_budget,
)

# Recombine the slices remaining after backpropagation
bp_circuit_trunc = combine_slices(
    remaining_slices_trunc, include_barriers=False
)

print(f"Backpropagated {metadata.num_backpropagated_slices} slices.")
print(
    f"New observable has {len(bp_obs_trunc.paulis)} terms, which can be combined into {len(bp_obs_trunc.group_commuting(qubit_wise=True))} groups.\n"
    f"After truncation, the error in our observable is bounded by {metadata.accumulated_error(0):.3e}"
)
print(
    f"Note that backpropagating one more slice would result in {metadata.backpropagation_history[-1].num_paulis[0]} terms "
    f"across {metadata.backpropagation_history[-1].num_qwc_groups} groups."
)
print("The remaining circuit after backpropagation looks as follows:")
bp_circuit_trunc.draw("mpl", fold=-1, scale=0.6)

Backpropagated 10 slices.
New observable has 646 terms, which can be combined into 14 groups.
After truncation, the error in our observable is bounded by 2.998e-02
Note that backpropagating one more slice would result in 1226 terms across 29 groups.
The remaining circuit after backpropagation looks as follows:


<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/c05a85bc-e5ca-4e02-8c96-98b28811f335-1.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/operator-back-propagation/extracted-outputs/b444d8bc-c800-4aa3-9927-eb807e92194f-1.avif)

## الجزء ب: توسيع النطاق!
لنستخدم الآن الانتشار العكسي للمؤثر (Operator Backpropagation) لدراسة ديناميكيات هاميلتوني سلسلة هايزنبرغ الكمومية (Heisenberg Spin Chain) المكوّنة من 50 كيوبت.
### الخطوة 1: تعيين المدخلات الكلاسيكية إلى مسألة كمومية
نأخذ في الاعتبار هاميلتوني مؤلف من 50 كيوبت، $\hat{\mathcal{H}}_{XYZ}$، للمسألة الموسّعة النطاق، مع الإبقاء على نفس قيم معاملات $J$ و$h$ المستخدمة في المثال الصغير النطاق. كما يبقى المرصود $M_Z = \frac{1}{N} \sum_{i=1}^N \langle Z_i \rangle$ كما هو. هذه المسألة تتجاوز قدرة المحاكاة الكلاسيكية العنيفة (brute-force simulation).

In [24]:
# Transpile original experiment
circuit_isa = pm.run(circuit)
observable_isa = observable.apply_layout(circuit_isa.layout)

# Transpile the backpropagated experiment
bp_circuit_isa = pm.run(bp_circuit)
bp_obs_isa = bp_obs_trunc.apply_layout(bp_circuit_isa.layout)

# Transpile the backpropagated experiment with truncated observable terms
bp_circuit_trunc_isa = pm.run(bp_circuit_trunc)
bp_obs_trunc_isa = bp_obs_trunc.apply_layout(bp_circuit_trunc_isa.layout)

In [25]:
print(
    f"2-qubit depth of original circuit: {circuit_isa.depth(lambda x:x.operation.num_qubits==2)}"
)
print(
    f"2-qubit depth of backpropagated circuit: {bp_circuit_isa.depth(lambda x:x.operation.num_qubits==2)}"
)
print(
    f"2-qubit depth of backpropagated circuit with truncation: {bp_circuit_trunc_isa.depth(lambda x:x.operation.num_qubits==2)}"
)

2-qubit depth of original circuit: 48
2-qubit depth of backpropagated circuit: 40
2-qubit depth of backpropagated circuit with truncation: 36


### Step 3: Execute using Qiskit primitives

In [26]:
pubs = [
    (circuit_isa, observable_isa),
    (bp_circuit_isa, bp_obs_isa),
    (bp_circuit_trunc_isa, bp_obs_trunc_isa),
]

In [27]:
options = EstimatorOptions()
options.default_precision = 0.01
options.resilience_level = 2
options.resilience.zne.noise_factors = [1, 1.2, 1.4]
options.resilience.zne.extrapolator = ["linear"]

estimator = EstimatorV2(mode=backend, options=options)

In [ ]:
job = estimator.run(pubs)

بالنسبة لهذه المسألة الموسّعة النطاق، اعتمدنا زمن تطور قدره $0.2$ مع $4$ خطوات trotterيّة. اختيرت هذه المسألة بحيث تتجاوز حدود المحاكاة الكلاسيكية العنيفة، غير أنها قابلة للمحاكاة بأسلوب شبكة الموترات (tensor network). يتيح لنا ذلك التحقق من النتيجة المحصّلة عبر الانتشار العكسي على حاسوب كمومي مقارنةً بالنتيجة المثالية.

القيمة المتوقعة المثالية لهذه المسألة، كما أُحصيت عبر محاكاة شبكة الموترات، هي $\simeq 0.89$.

In [ ]:
result_no_bp = job.result()[0].data.evs.item()
result_bp = job.result()[1].data.evs.item()
result_bp_trunc = job.result()[2].data.evs.item()

In [ ]:
print(f"Expectation value without backpropagation: {result_no_bp}")
print(f"Backpropagated expectation value: {result_bp}")
print(f"Backpropagated expectation value with truncation: {result_bp_trunc}")

Expectation value without backpropagation: 0.7887194658035515
Backpropagated expectation value: 0.9532818300978584
Backpropagated expectation value with truncation: 0.8913400398926913


In [ ]:
methods = [
    "No backpropagation",
    "Backpropagation",
    "Backpropagation w/ truncation",
]
values = [result_no_bp, result_bp, result_bp_trunc]

ax = plt.gca()
plt.bar(methods, values, color="#a56eff", width=0.4, edgecolor="#8a3ffc")
plt.axhline(0.89)
ax.set_ylim([0.6, 0.98])
plt.text(0.2, 0.895, "Exact result")
ax.set_ylabel(r"$M_Z$", fontsize=12)

Text(0, 0.5, '$M_Z$')

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/047d448f-aebf-45ff-a81b-83b2d5ca866d-1.avif" alt="Output of the previous code cell" />

نحدد `max_error_per_slice` بقيمة 0.005 كما في السابق. غير أن عدد الشرائح في هذه المسألة الكبيرة النطاق أعلى بكثير مما كان في المسألة الصغيرة، مما يعني أن السماح بخطأ 0.005 لكل شريحة قد يُفضي إلى خطأ إجمالي كبير في الانتشار العكسي. يمكن تقييد ذلك بتحديد `max_error_total` الذي يحدّ من إجمالي خطأ الانتشار العكسي، وقد حددنا قيمته بـ 0.03 (وهو تقريبا ما كان عليه في المثال الصغير النطاق).

في هذا المثال الكبير النطاق، نسمح بقيمة أعلى لعدد المجموعات المتبادلة التبديل (commuting groups)، إذ نضبطه على 15.